In [ ]:

## Load train data
v3data.train <- read.csv(...)

#### Step 1.  Remember to check multicolineatity to remove highly correlated variables 

# Define stepwise VIF function 
vif_func<-function(in_frame,thresh=10,trace=T,...){

  library(fmsb)
  
  if(any(!'data.frame' %in% class(in_frame))) in_frame<-data.frame(in_frame)
  
  #get initial vif value for all comparisons of variables
  vif_init<-NULL
  var_names <- names(in_frame)
  for(val in var_names){
      regressors <- var_names[-which(var_names == val)]
      form <- paste(regressors, collapse = '+')
      form_in <- formula(paste(val, '~', form))
      vif_init<-rbind(vif_init, c(val, VIF(lm(form_in, data = in_frame, ...))))
      }
  vif_max<-max(as.numeric(vif_init[,2]), na.rm = TRUE)

  if(vif_max < thresh){
    if(trace==T){ #print output of each iteration
        prmatrix(vif_init,collab=c('var','vif'),rowlab=rep('',nrow(vif_init)),quote=F)
        cat('\n')
        cat(paste('All variables have VIF < ', thresh,', max VIF ',round(vif_max,2), sep=''),'\n\n')
        }
    return(var_names)
    }
  else{

    in_dat<-in_frame

    #backwards selection of explanatory variables, stops when all VIF values are below 'thresh'
    while(vif_max >= thresh){
      
      vif_vals<-NULL
      var_names <- names(in_dat)
        
      for(val in var_names){
        regressors <- var_names[-which(var_names == val)]
        form <- paste(regressors, collapse = '+')
        form_in <- formula(paste(val, '~', form))
        vif_add<-VIF(lm(form_in, data = in_dat, ...))
        vif_vals<-rbind(vif_vals,c(val,vif_add))
        }
      max_row<-which(vif_vals[,2] == max(as.numeric(vif_vals[,2]), na.rm = TRUE))[1]

      vif_max<-as.numeric(vif_vals[max_row,2])

      if(vif_max<thresh) break
      
      if(trace==T){ #print output of each iteration
        prmatrix(vif_vals,collab=c('var','vif'),rowlab=rep('',nrow(vif_vals)),quote=F)
        cat('\n')
        cat('removed: ',vif_vals[max_row,1],vif_max,'\n\n')
        flush.console()
        }

      in_dat<-in_dat[,!names(in_dat) %in% vif_vals[max_row,1]]

      }

    return(names(in_dat))
    
    }
  
  }

# Check VIF

df.subset <- subset(v3data.train, select= -c(Log_new_cases_per_million, weather_situation, isHoliday, is_weekend, Day_of_Week, continent))

vif_result = vif_func(in_frame=df.subset ,thresh=10,trace=T)

# Remove variable with VIF > 10 from the dataset v3data.train

#### Step 2: Feature selection by Boruta
 
library(Boruta)
set.seed(123)
boruta.train <- Boruta(Log_new_cases_per_million ~. , data = v3data.train, doTrace = 2)

print(boruta.train)

plot(boruta.train, xlab = "", xaxt = "n")
lz<-lapply(1:ncol(boruta.train$ImpHistory),function(i)
boruta.train$ImpHistory[is.finite(boruta.train$ImpHistory[,i]),i])
names(lz) <- colnames(boruta.train$ImpHistory)
Labels <- sort(sapply(lz,median))
axis(side = 1,las=2,labels = names(Labels),
at = 1:ncol(boruta.train$ImpHistory), cex.axis = 0.7)


final.boruta <- TentativeRoughFix(boruta.train)
print(final.boruta)

getSelectedAttributes(final.boruta, withTentative = F)

boruta.df <- attStats(final.boruta)
class(boruta.df)
print(boruta.df)


#### Step 3: Feature selection by RFE

library(caret)
library(randomForest)
library(doParallel)
Mycluster = makeCluster(detectCores())
registerDoParallel(Mycluster)
set.seed(143)
control <- rfeControl(functions=rfFuncs, method="repeatedcv", number=10, repeats=3, allowParallel = TRUE)
rfe.train <- rfe(v3data.train[,1:78],v3data.train[,79], sizes=1:78, rfeControl=control) 
# v3data.train[,1:78] refers all input variables, v3data.train[,79] refers output variable

rfe.train
plot(rfe.train, type=c("g", "o"), cex = 1.0, col = 1:11)
predictors(rfe.train)